# **INTRODUCTIONS**

---

Used for exceptional processes.

The code is separated into multiple sections:

1. **[VACCINATION_CAMPAIGNS_20220908](#vaccination-campaigns-20220908)**
2. **[VACCINATION_SHOTS_PLANNED_20220908](#vaccination-shots-planned-20220908)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **VACCINATION CAMPAIGNS 20220908**

---

In [ ]:
DROP TABLE IF EXISTS VWSARCHIVE.RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908

CREATE TABLE VWSARCHIVE.RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908 (
    VACCINE_CAMPAIGN_ORDER INT NULL,
    VACCINE_CAMPAIGN_NAME_NL nvarchar(250) NULL,
    VACCINE_CAMPAIGN_NAME_EN nvarchar(250) NULL,
    VACCINE_ADMINISTERED_TOTAL INT NULL,
    VACCINE_ADMINISTERED_LAST_WEEK INT NULL,
    DATE_UNIX BIGINT NULL,
    DATE_END_UNIX BIGINT NULL,
    DATE_START_UNIX BIGINT NULL,
    DATE_OF_INSERTION_UNIX BIGINT NULL
);

In [ ]:
-- Hiervoor eenmalig een query runnen die dit INTO een VWSARCHIVE tabel schrijft
-- met als extensie 20220908 en die filtert op de datum 2022-0908


-- 1) CREATE STORED PROCEDURE(S): ARCHIVE EXISTING DATA
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908]
AS
BEGIN

TRUNCATE TABLE [VWSARCHIVE].[RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908];

WITH CTE AS (
    SELECT
            [DATE_LAST_INSERTED],
            [DATE_OF_REPORT],
            [DATE_OF_STATISTICS],
            [CAMPAIGN],
            [TOTAL_VACCINATED],
            [CUMSUM_TOTAL_VACCINATED],
            [TOTAL_VACCINATED_7DAYS]
    FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL])
    )
INSERT INTO VWSARCHIVE.RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908 (
    VACCINE_CAMPAIGN_ORDER,
    VACCINE_CAMPAIGN_NAME_NL,
    VACCINE_CAMPAIGN_NAME_EN,
    VACCINE_ADMINISTERED_TOTAL,
    VACCINE_ADMINISTERED_LAST_WEEK,
    DATE_UNIX,
    DATE_END_UNIX,
    DATE_START_UNIX,
    DATE_OF_INSERTION_UNIX 
)
SELECT
            r.[ORDER] AS [VACCINE_CAMPAIGN_ORDER],
            r.[CAMPAIGN_LABEL_NL] AS [VACCINE_CAMPAIGN_NAME_NL],
            r.[CAMPAIGN_LABEL_EN] AS [VACCINE_CAMPAIGN_NAME_EN],
            c.[CUMSUM_TOTAL_VACCINATED] AS [VACCINE_ADMINISTERED_TOTAL],
            c.[TOTAL_VACCINATED_7DAYS] AS [VACCINE_ADMINISTERED_LAST_WEEK],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_START_ISO](c.[DATE_OF_STATISTICS])) AS [DATE_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_START_ISO](c.[DATE_OF_STATISTICS])) AS [DATE_END_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX](dateadd(day, -6,  [dbo].[WEEK_START_ISO](c.[DATE_OF_STATISTICS]))) AS [DATE_START_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX](c.[DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
FROM CTE c
INNER JOIN (
    SELECT
            [CAMPAIGN_CODE],
            [CAMPAIGN_LABEL_NL],
            [CAMPAIGN_LABEL_EN],
            [ORDER]
    FROM [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] WITH(NOLOCK))
    ) r ON c.CAMPAIGN = r.[CAMPAIGN_CODE]
WHERE c.[DATE_OF_STATISTICS] = (SELECT MAX([DATE_OF_STATISTICS]) FROM  CTE)
END;
GO

In [ ]:
-- View aanmaken die data ophaalt en die als bron kan dienen voor generatie van Protos
CREATE OR ALTER View VWSARCHIVE.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908
AS
SELECT * FROM VWSARCHIVE.RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908

GO

In [ ]:
-- View aanmaken in DataTino_Protos_1.Views
DECLARE @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSARCHIVE.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908%')
DELETE FROM DATATINO_PROTO_1.Configurations WHERE View_ID = @ViewID
DELETE FROM Datatino_Proto_1.Views WHERE ID = @ViewID

INSERT INTO Datatino_Proto_1.Views (Last_Update_name, Constraint_Key_name, Constraint_Value, Grouped_Key_Name, Grouped_Last_Update_Name, Name, Description)
SELECT 'DATE_UNIX',
       NULL,
       NULL,
       NULL,
       NULL,
       'VWSARCHIVE.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908',
       'VIEW: VWSDEST.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908 FOR vaccine_campaigns_archived_20220908/vaccine_campaigns_archived_20220908'

In [ ]:
-- Twee regels toevoegen aan Datatino_Proto_1.Configurations
DECLARE @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSARCHIVE.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908%')
DECLARE @ProtoID INT = (SELECT ID FROM Datatino_Proto_1.Protos WHERE Name = 'NL')

-- Eerst evt al aanwezige regels verwijderen
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE Proto_ID = @ProtoID and View_ID = @ViewID

-- Regel toevoegen voor 'header'
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0, 
       'DATE_UNIX|DATE_END_UNIX|DATE_START_UNIX|DATE_OF_INSERTION_UNIX',
       '=LOWER()',
       3,
       NULL,
       1,
       'vaccine_campaigns_archived_20220908',
       'VIEW CONFIGURATION: VWSARCHIVE.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908 FOR vaccine_campaigns_archived_20220908'

-- Regel toevoegen voor 'details'
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0,
       'VACCINE_CAMPAIGN_ORDER|VACCINE_CAMPAIGN_NAME_NL|VACCINE_CAMPAIGN_NAME_EN|VACCINE_ADMINISTERED_TOTAL|VACCINE_ADMINISTERED_LAST_WEEK',
       '=LOWER()',
       4,
       NULL,
       1,
       'vaccine_campaigns_archived_20220908/vaccine_campaigns',
       'VIEW CONFIGURATION: VWSARCHIVE.V_RIVM_VACCINATION_CAMPAIGNS_NL_ARCHIVED_20220908 FOR vaccine_campaigns_archived_20220908/vaccine_campaigns_archived_20220908'
       


# **VACCINATION SHOTS PLANNED 20220908**

---

In [ ]:
DROP TABLE IF EXISTS VWSARCHIVE.RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908
CREATE TABLE VWSARCHIVE.RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908 (
    DOSES INT NULL,
    DATE_UNIX BIGINT NULL,
    DATE_START_UNIX BIGINT NULL,
    DATE_END_UNIX BIGINT NULL,
    DATE_OF_INSERTION_UNIX BIGINT
);

In [ ]:
-- Hiervoor eenmalig een query runnen die dit INTO een VWSARCHIVE tabel schrijft
-- met als extensie _ARCHIVED_20220908 en die filtert op de datum 2022-0908

-- 1) CREATE STORED PROCEDURE(S): ARCHIVE EXISTING DATA
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908]
AS
BEGIN

TRUNCATE TABLE [VWSARCHIVE].[RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908];

WITH CTE AS (
    SELECT TOP(1)
        [DOSES],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS_START]) AS [DATE_START_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS_END]) AS [DATE_END_UNIX],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
    FROM 
        [VWSDEST].[RIVM_VACCINATION_SHOTS_PLANNED_NL] WITH(NOLOCK)
    WHERE CAST([DATE_LAST_INSERTED] as Date) = CAST('20220908' as DATE)
)
INSERT INTO [VWSARCHIVE].[RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908] (
    DOSES,
    DATE_END_UNIX,
    DATE_START_UNIX,
    DATE_UNIX,
    DATE_OF_INSERTION_UNIX
)
SELECT
    [DOSES],
    [DATE_END_UNIX],
    [DATE_START_UNIX],
    [DATE_START_UNIX] AS [DATE_UNIX],
    [DATE_OF_INSERTION_UNIX]
FROM CTE

END;
GO

In [ ]:
-- View aanmaken die data ophaalt en die als bron kan dienen voor generatie van Protos
CREATE OR ALTER View VWSARCHIVE.V_RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908
AS
SELECT * FROM VWSARCHIVE.RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908

GO

In [ ]:
-- View aanmaken in DataTino_Protos_1.Views
DECLARE @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSARCHIVE.V_RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908%')
DELETE FROM Datatino_Proto_1.Configurations WHERE View_ID = @ViewID
DELETE FROM Datatino_Proto_1.Views WHERE ID = @ViewID

INSERT INTO Datatino_Proto_1.Views (Last_Update_name, Constraint_Key_name, Constraint_Value, Grouped_Key_Name, Grouped_Last_Update_Name, Name, Description)
SELECT 'DATE_UNIX',
       NULL,
       NULL,
       NULL,
       NULL,
       'VWSARCHIVE.V_RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908',
       'VIEW: VWSARCHIVE.V_RIVM_VACCINATION_SHOTS_PLANNED_ARCHIVED_20220908 FOR vaccine_planned_archived_20220908/vaccine_planned_archived_20220908'

In [ ]:
-- Regel toevoegen aan Datatino_Proto_1.Configurations
DECLARE @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSARCHIVE.V_RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908%')
DECLARE @ProtoID INT = (SELECT ID FROM Datatino_Proto_1.Protos WHERE Name = 'NL')

-- Eerst evt al aanwezige regels verwijderen
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE Proto_ID = @ProtoID and View_ID = @ViewID

-- Regel toevoegen 
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0, 
       '*',
       '=LOWER()',
       3,
       NULL,
       1,
       'vaccine_planned_archived_20220908',
       'VIEW CONFIGURATION: VWSARCHIVE.V_RIVM_VACCINATION_SHOTS_PLANNED_NL_ARCHIVED_20220908 FOR vaccine_planned_archived_20220908'